In [1]:
from vae import encoder, generator
import keras.backend as K
from keras.layers import Lambda, Dense as kDense, Multiply, PReLU, ZeroPadding1D, Input, Concatenate, Reshape
from keras.models import Model
from keras.optimizers import Adam
from keras.objectives import mean_squared_error
from models_keras import MADE, _mask_matrix_made, Dense, _activation, gate, Conv1D, Dense
import numpy as np
from parse_dataset import *
from sklearn.preprocessing import StandardScaler

Using TensorFlow backend.


# Load and parse dataset

In [2]:
data = pd.read_csv('Sepsis_imp.csv')
replace_absurd_temperatures(data)
data = drop_patients_with_absurd_weights(data)
data = drop_patients_with_unrealistic_HR_or_BP(data)
data = add_relative_time_column(data)
data = drop_patient_with_negative_input(data)
add_small_quantities(data)
create_action_column(data)
add_log_actions(data)

log_scaler = StandardScaler()
scaler = StandardScaler()
action_scaler = StandardScaler()
train_idx, test_idx = split_train_test_idx(data)

# scale on train data only
scaler.fit(data.loc[data.icustayid.isin(train_idx)][numerical_columns_not_to_be_logged])
log_scaler.fit(np.log(data.loc[data.icustayid.isin(train_idx)][numerical_columns_to_be_logged]))
action_scaler.fit(data.loc[data.icustayid.isin(train_idx)][log_action_cols])

StandardScaler(copy=True, with_mean=True, with_std=True)

In [3]:
def data_generator(data, last, idx, scaler, log_scaler, action_scaler, batch_size=32, log_action=True, verbose=False):
    """used in fit_generator"""
    X_action_bin, X_features_bin, X_num, X_action, X_finished, X_alive = matrify(data, idx, scaler, log_scaler,action_scaler, verbose=False, log_action=log_action)
    while True:
        # shuffle
        X_action_bin, X_features_bin, X_num, X_action, X_finished, X_alive = shuffle(X_action_bin, X_features_bin, X_num, X_action, X_finished, X_alive)

        # take a part only
        if last is not None:
            X_action_bin, X_features_bin, X_num, X_action, X_finished, X_alive = X_action_bin[:last], X_features_bin[:last], X_num[:last], X_action[:last], X_finished[:last], X_alive[:last]

        # one-hot encode
        if verbose:
            iterator = tqdm(range(0, X_num.shape[0], batch_size))
        else:
            iterator = range(0, X_num.shape[0], batch_size)
        for i in iterator:
            x_action_bin = X_action_bin[i:i+batch_size]
            x_features_bin = X_features_bin[i:i+batch_size]
            x_num = X_num[i:i+batch_size]
            action = X_action[i:i+batch_size]
            finished = X_finished[i:i+batch_size]
            alive = X_alive[i:i+batch_size]
            
            if x_num.shape[0] != batch_size:
                continue
            yield [x_num, x_features_bin, x_action_bin, action], [x_num, finished, alive]

# Build model

In [4]:
E = encoder(num_filters=50)
G = generator()

In [5]:
def sample_z(batch_size, latent_dim, epsilon_std):
    return lambda args: _sampling(args, batch_size, latent_dim, epsilon_std)

def _sampling(args, batch_size, latent_dim, epsilon_std):
    z_mean, z_var = args
    epsilon = K.random_normal(shape=(batch_size, latent_dim), mean=0.,
                              stddev=epsilon_std)
    return z_mean + K.sqrt(z_var+1e-8) * epsilon

In [6]:
batch_size = 128
latent_dim = 32

In [7]:
# x -> z
vitals, binary_features, binary_actions, actions = E.inputs
z_mean, z_var = E([vitals, binary_features, binary_actions, actions])  # we need those to compute KL divergence, so it cannot be directly included in the encoder function
z = Lambda(sample_z(batch_size, latent_dim, 1.), output_shape=(latent_dim,))([z_mean, z_var])

E = Model([vitals, binary_features, binary_actions, actions], z)

In [8]:
E.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_3 (InputLayer)             (None, 32, 45)        0                                            
____________________________________________________________________________________________________
input_1 (InputLayer)             (None, 2)             0                                            
____________________________________________________________________________________________________
input_2 (InputLayer)             (None, 32, 3)         0                                            
____________________________________________________________________________________________________
input_4 (InputLayer)             (None, 32, 2)         0                                            
___________________________________________________________________________________________

In [9]:
# x -> z -> x'
# update the encoder and generator weights on their ability to reconstruct the input protein
vitals, binary_features, binary_actions, actions = E.inputs
z = E([vitals, binary_features, binary_actions, actions])

next_vitals, finished, dead_or_alive = G([vitals, binary_features, binary_actions, actions, z])
VAE = Model([vitals, binary_features, binary_actions, actions], [next_vitals, finished, dead_or_alive])

def _vae_loss(x, x_decoded_mean, z_mean, z_var):
    mse = mean_squared_error(x, x_decoded_mean)
    mse_loss = K.sum(mse, -1)
    kl_loss = - 0.5 * K.sum(1 + K.log(z_var+1e-8) - K.square(z_mean) - z_var, axis=-1)
#     return K.mean(mse_loss + kl_loss)
    return mse_loss + kl_loss
def vae_loss(z_mean, z_var):
    return lambda x, x_decoded: _vae_loss(x, x_decoded, z_mean, z_var)

VAE.compile(loss=[vae_loss(z_mean, z_var), 'binary_crossentropy', 'binary_crossentropy'], optimizer=Adam())

In [10]:
VAE.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_3 (InputLayer)             (None, 32, 45)        0                                            
____________________________________________________________________________________________________
input_1 (InputLayer)             (None, 2)             0                                            
____________________________________________________________________________________________________
input_2 (InputLayer)             (None, 32, 3)         0                                            
____________________________________________________________________________________________________
input_4 (InputLayer)             (None, 32, 2)         0                                            
___________________________________________________________________________________________

# Train model

In [ ]:
epochs = 10
last_test = ((test_idx.shape[0] // batch_size)*batch_size) - test_idx.shape[0]
last_test = test_idx.shape[0] if last_test == 0 else last_test
last_train = ((train_idx.shape[0] // batch_size)*batch_size) - train_idx.shape[0]
last_train = train_idx.shape[0] if last_train == 0 else last_train

h = VAE.fit_generator(
    data_generator(data, last_train, train_idx, scaler, log_scaler, action_scaler, batch_size=batch_size, verbose=False),
    train_idx[:last_train].shape[0]/batch_size, 
    verbose=1,
    epochs=epochs,
    validation_data=data_generator(data, last_test, test_idx, scaler, log_scaler, action_scaler, batch_size=batch_size),
    validation_steps=test_idx[:last_test].shape[0]/batch_size
)

Epoch 1/10


 80/141 [================>.............] - ETA: 8586s - loss: 40.1307 - model_2_loss_1: 38.3853 - model_2_loss_2: 0.9231 - model_2_loss_3: 0.822 - ETA: 4291s - loss: 39.8566 - model_2_loss_1: 38.1234 - model_2_loss_2: 0.9153 - model_2_loss_3: 0.818 - ETA: 2859s - loss: 39.8386 - model_2_loss_1: 38.1382 - model_2_loss_2: 0.8900 - model_2_loss_3: 0.810 - ETA: 2143s - loss: 39.6810 - model_2_loss_1: 38.0028 - model_2_loss_2: 0.8695 - model_2_loss_3: 0.808 - ETA: 1713s - loss: 39.4325 - model_2_loss_1: 37.7767 - model_2_loss_2: 0.8497 - model_2_loss_3: 0.806 - ETA: 1426s - loss: 39.3303 - model_2_loss_1: 37.7029 - model_2_loss_2: 0.8266 - model_2_loss_3: 0.800 - ETA: 1221s - loss: 39.1403 - model_2_loss_1: 37.5320 - model_2_loss_2: 0.8087 - model_2_loss_3: 0.799 - ETA: 1067s - loss: 39.1620 - model_2_loss_1: 37.5759 - model_2_loss_2: 0.7916 - model_2_loss_3: 0.794 - ETA: 947s - loss: 39.0060 - model_2_loss_1: 37.4406 - model_2_loss_2: 0.7735 - model_2_loss_3: 0.791 - ETA: 851s - loss: 38.9

 80/141 [================>.............] - ETA: 58s - loss: 25.7925 - model_2_loss_1: 24.7725 - model_2_loss_2: 0.3707 - model_2_loss_3: 0.649 - ETA: 61s - loss: 25.8025 - model_2_loss_1: 24.7915 - model_2_loss_2: 0.3719 - model_2_loss_3: 0.639 - ETA: 60s - loss: 25.8254 - model_2_loss_1: 24.8098 - model_2_loss_2: 0.3720 - model_2_loss_3: 0.643 - ETA: 62s - loss: 25.9832 - model_2_loss_1: 24.9703 - model_2_loss_2: 0.3736 - model_2_loss_3: 0.639 - ETA: 63s - loss: 26.0243 - model_2_loss_1: 25.0106 - model_2_loss_2: 0.3744 - model_2_loss_3: 0.639 - ETA: 61s - loss: 25.9710 - model_2_loss_1: 24.9587 - model_2_loss_2: 0.3743 - model_2_loss_3: 0.638 - ETA: 60s - loss: 25.9502 - model_2_loss_1: 24.9379 - model_2_loss_2: 0.3747 - model_2_loss_3: 0.637 - ETA: 58s - loss: 25.9006 - model_2_loss_1: 24.8884 - model_2_loss_2: 0.3746 - model_2_loss_3: 0.637 - ETA: 57s - loss: 25.8590 - model_2_loss_1: 24.8459 - model_2_loss_2: 0.3741 - model_2_loss_3: 0.639 - ETA: 56s - loss: 25.8184 - model_2_loss

141/141 [==============================] - ETA: 26s - loss: 23.8250 - model_2_loss_1: 22.8433 - model_2_loss_2: 0.3598 - model_2_loss_3: 0.621 - ETA: 25s - loss: 23.7994 - model_2_loss_1: 22.8182 - model_2_loss_2: 0.3597 - model_2_loss_3: 0.621 - ETA: 25s - loss: 23.7713 - model_2_loss_1: 22.7904 - model_2_loss_2: 0.3596 - model_2_loss_3: 0.621 - ETA: 24s - loss: 23.7445 - model_2_loss_1: 22.7640 - model_2_loss_2: 0.3593 - model_2_loss_3: 0.621 - ETA: 24s - loss: 23.7216 - model_2_loss_1: 22.7415 - model_2_loss_2: 0.3592 - model_2_loss_3: 0.620 - ETA: 24s - loss: 23.6942 - model_2_loss_1: 22.7145 - model_2_loss_2: 0.3590 - model_2_loss_3: 0.620 - ETA: 23s - loss: 23.6684 - model_2_loss_1: 22.6894 - model_2_loss_2: 0.3588 - model_2_loss_3: 0.620 - ETA: 23s - loss: 23.6401 - model_2_loss_1: 22.6618 - model_2_loss_2: 0.3586 - model_2_loss_3: 0.619 - ETA: 22s - loss: 23.6171 - model_2_loss_1: 22.6393 - model_2_loss_2: 0.3583 - model_2_loss_3: 0.619 - ETA: 22s - loss: 23.5905 - model_2_loss

 80/141 [================>.............] - ETA: 55s - loss: 19.1483 - model_2_loss_1: 18.2344 - model_2_loss_2: 0.3268 - model_2_loss_3: 0.587 - ETA: 57s - loss: 18.9533 - model_2_loss_1: 18.0439 - model_2_loss_2: 0.3281 - model_2_loss_3: 0.581 - ETA: 55s - loss: 18.9616 - model_2_loss_1: 18.0438 - model_2_loss_2: 0.3287 - model_2_loss_3: 0.589 - ETA: 55s - loss: 19.0219 - model_2_loss_1: 18.1065 - model_2_loss_2: 0.3286 - model_2_loss_3: 0.586 - ETA: 55s - loss: 19.0033 - model_2_loss_1: 18.0925 - model_2_loss_2: 0.3284 - model_2_loss_3: 0.582 - ETA: 55s - loss: 19.0603 - model_2_loss_1: 18.1477 - model_2_loss_2: 0.3293 - model_2_loss_3: 0.583 - ETA: 54s - loss: 19.0009 - model_2_loss_1: 18.0858 - model_2_loss_2: 0.3293 - model_2_loss_3: 0.585 - ETA: 54s - loss: 18.9624 - model_2_loss_1: 18.0459 - model_2_loss_2: 0.3301 - model_2_loss_3: 0.586 - ETA: 54s - loss: 18.9590 - model_2_loss_1: 18.0447 - model_2_loss_2: 0.3300 - model_2_loss_3: 0.584 - ETA: 53s - loss: 18.9113 - model_2_loss

 80/141 [================>.............] - ETA: 65s - loss: 13.4346 - model_2_loss_1: 12.6181 - model_2_loss_2: 0.2887 - model_2_loss_3: 0.527 - ETA: 63s - loss: 13.2490 - model_2_loss_1: 12.4322 - model_2_loss_2: 0.2920 - model_2_loss_3: 0.524 - ETA: 63s - loss: 13.2086 - model_2_loss_1: 12.3915 - model_2_loss_2: 0.2926 - model_2_loss_3: 0.524 - ETA: 61s - loss: 13.1241 - model_2_loss_1: 12.3039 - model_2_loss_2: 0.2914 - model_2_loss_3: 0.528 - ETA: 61s - loss: 13.2508 - model_2_loss_1: 12.4306 - model_2_loss_2: 0.2918 - model_2_loss_3: 0.528 - ETA: 61s - loss: 13.2355 - model_2_loss_1: 12.4114 - model_2_loss_2: 0.2915 - model_2_loss_3: 0.532 - ETA: 61s - loss: 13.2117 - model_2_loss_1: 12.3870 - model_2_loss_2: 0.2917 - model_2_loss_3: 0.533 - ETA: 60s - loss: 13.1793 - model_2_loss_1: 12.3552 - model_2_loss_2: 0.2913 - model_2_loss_3: 0.532 - ETA: 60s - loss: 13.1372 - model_2_loss_1: 12.3133 - model_2_loss_2: 0.2908 - model_2_loss_3: 0.533 - ETA: 60s - loss: 13.1043 - model_2_loss

140/141 [============================>.] - ETA: 28s - loss: 11.2623 - model_2_loss_1: 10.4478 - model_2_loss_2: 0.2869 - model_2_loss_3: 0.527 - ETA: 28s - loss: 11.2252 - model_2_loss_1: 10.4109 - model_2_loss_2: 0.2869 - model_2_loss_3: 0.527 - ETA: 27s - loss: 11.1887 - model_2_loss_1: 10.3744 - model_2_loss_2: 0.2869 - model_2_loss_3: 0.527 - ETA: 27s - loss: 11.1502 - model_2_loss_1: 10.3359 - model_2_loss_2: 0.2869 - model_2_loss_3: 0.527 - ETA: 27s - loss: 11.1125 - model_2_loss_1: 10.2982 - model_2_loss_2: 0.2869 - model_2_loss_3: 0.527 - ETA: 26s - loss: 11.0841 - model_2_loss_1: 10.2700 - model_2_loss_2: 0.2869 - model_2_loss_3: 0.527 - ETA: 26s - loss: 11.0488 - model_2_loss_1: 10.2346 - model_2_loss_2: 0.2869 - model_2_loss_3: 0.527 - ETA: 25s - loss: 11.0090 - model_2_loss_1: 10.1950 - model_2_loss_2: 0.2869 - model_2_loss_3: 0.527 - ETA: 25s - loss: 10.9664 - model_2_loss_1: 10.1524 - model_2_loss_2: 0.2869 - model_2_loss_3: 0.527 - ETA: 24s - loss: 10.9277 - model_2_loss

In [ ]:
X_action_bin, X_features_bin, X_num, X_action, X_finished, X_alive = matrify(data, test_idx, scaler, log_scaler,action_scaler)

In [ ]:
X_action_bin, X_features_bin, X_num, X_action, X_finished, X_alive = X_action_bin[:last_test], X_features_bin[:last_test], X_num[:last_test], X_action[:last_test], X_finished[:last_test], X_alive[:last_test]

`vitals, binary_features, binary_actions, actions`

In [ ]:
X_recons, finished, dead_or_alive = VAE.predict([X_num, X_features_bin, X_action_bin, X_action])

In [ ]:
X_num[0][15:]

In [ ]:
for feature, error in zip(numerical_columns_not_to_be_logged+numerical_columns_to_be_logged, np.mean((X_recons[0] - X_num[0])[:16]**2, 0)):
    print(feature, error)

In [ ]:
np.sqrt(0.23)